Treinando uma rede neural usando pytorch
========================================



## Introdução



O notebook anterior foi o *season finale* da construção da rede neural usando Python puro. Este notebook é o epílogo.

Aqui veremos como podemos construir e treinar uma rede neural usando `pytorch`, um módulo especializado em redes neurais artificiais.

Motivos para usarmos o `pytorch`:

1.  A `MLP` que criamos em Python puro não é otimizada. Tente treinar uma rede com uns 100 neurônios em uma camada e verá como ela é bem lenta! `pytorch` é escrito por diversos programadores que além de primar pelas contas estarem corretas, eles fazem de tudo para que elas ocorram de forma eficiente.

2.  `pytorch` já tem praticamente tudo que precisamos implementado! Quer calcular o gradiente local da função arco tangente? Ele já tem! Quer usar a função de ativação ReLU? Ele já tem!

3.  `pytorch` tem suporte a treinar uma rede neural usando processamento gráfico (GPU). Isso acelera muito o treino de redes neurais artificiais complexas. Temos GPUs para usarmos no HPC da Ilum!

4.  `pytorch` é usado tanto na academia quanto no mundo corporativo. Junto com `tensorflow` são os dois módulos estado da arte para o projeto e treino de redes neurais.



## Objetivo



Treinar uma rede neural artificial tipo Multilayer Perceptron usando `pytorch`.



## Importações



In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error

## Código e discussão



### Dados



Vamos treinar nossa rede neural com nosso velho amigo, o dataset de diamantes! Como todo processo de treino de aprendizado de máquina, precisamos reservar um conjunto de dados para treino e outro para teste.



In [2]:
TAMANHO_TESTE = 0.1
SEMENTE_ALEATORIA = 61455
DATASET_NAME = "diamonds"
FEATURES = ["carat", "depth", "table", "x", "y", "z"]
TARGET = ["price"]

df = sns.load_dataset(DATASET_NAME)

indices = df.index
indices_treino, indices_teste = train_test_split(
    indices, test_size=TAMANHO_TESTE, random_state=SEMENTE_ALEATORIA
)

df_treino = df.loc[indices_treino]
df_teste = df.loc[indices_teste]

X_treino = df_treino.reindex(FEATURES, axis=1)
y_treino = df_treino.reindex(TARGET, axis=1)
X_teste = df_teste.reindex(FEATURES, axis=1)
y_teste = df_teste.reindex(TARGET, axis=1)

Redes neurais costumam se dar bem quando os dados estão entre $[0, 1]$ ou entre $[-1, 1]$. Redes neurais costumam não se dar bem quando os dados estão em escalas muito diferentes. Por conta disso, normalizar os dados antes de prosseguir é recomendado! Vamos usar o `MinMaxScaler` do `scikit-learn` que aprendemos semestre passado.



In [3]:
normalizador_x = MinMaxScaler()
normalizador_y = MinMaxScaler()

normalizador_x.fit(X_treino)
normalizador_y.fit(y_treino)

X_treino = normalizador_x.transform(X_treino)
y_treino = normalizador_y.transform(y_treino)
X_teste = normalizador_x.transform(X_teste)
y_teste = normalizador_y.transform(y_teste)

### Tensores



Na nossa rede neural feita em Python puro, a base de tudo era a classe `Valor`. Para o `pytorch`, a base de tudo são os tensores! Tensores são objetos que armazenam dados de forma similar aos arrays de `numpy`, porém registram todas as operações realizadas para o cálculo do gradiente local (assim como nós fizemos com a classe `Valor`).



In [4]:
X_treino = torch.tensor(X_treino, dtype=torch.float32)
y_treino = torch.tensor(y_treino, dtype=torch.float32)
X_teste = torch.tensor(X_teste, dtype=torch.float32)
y_teste = torch.tensor(y_teste, dtype=torch.float32)

Vamos checar os dados para ver como estão.



In [5]:
print(X_treino)
print()
print(y_treino)

tensor([[0.0686, 0.5611, 0.2885, 0.4777, 0.0876, 0.1022],
        [0.0624, 0.5250, 0.2500, 0.4730, 0.0871, 0.0994],
        [0.0478, 0.4917, 0.2692, 0.4572, 0.0829, 0.0934],
        ...,
        [0.1185, 0.5333, 0.2692, 0.5410, 0.0995, 0.1142],
        [0.0166, 0.5139, 0.2885, 0.3929, 0.0708, 0.0811],
        [0.0333, 0.5361, 0.2692, 0.4209, 0.0776, 0.0890]])

tensor([[0.0676],
        [0.0586],
        [0.0434],
        ...,
        [0.1527],
        [0.0172],
        [0.0216]])


### Criando a rede neural



Para criar uma rede neural usando `pytorch` é necessário criar uma classe. Observe aqui que a classe criada é baseada na classe `nn.Module` do `pytorch`. Classes podem ser baseadas em outras classes! Trata-se da característica de herança das classes.

Observe que definimos as camadas da rede neural dentro de um objeto `nn.Sequential`.

Observe também o método `forward`. Este funciona de forma similar ao dunder `__call__` que vimos anteriormente. O `pytorch` requer que usemos o `forward` e não o `__call__`, então aceitamos e seguimos em frente.



In [6]:
class MLP(nn.Module):
    def __init__(
        self, num_dados_entrada, neuronios_c1, neuronios_c2, num_targets
    ):
        # Temos que inicializar a classe mãe
        super().__init__()

        # Definindo as camadas da rede
        self.camadas = nn.Sequential(
                    nn.Linear(num_dados_entrada, neuronios_c1),
                    nn.ReLU(),
                    nn.Linear(neuronios_c1, neuronios_c2),
                    nn.ReLU(),
                    nn.Linear(neuronios_c2, num_targets),
                )

    def forward(self, x):
        """Esse é o método que executa a rede do pytorch."""
        x = self.camadas(x)
        return x

Agora podemos criar a nossa rede neural!



In [7]:
NUM_DADOS_DE_ENTRADA = X_treino.shape[1]
NUM_DADOS_DE_SAIDA = y_treino.shape[1]
NEURONIOS_C1 = 50
NEURONIOS_C2 = 20

minha_MLP = MLP(NUM_DADOS_DE_ENTRADA, NEURONIOS_C1, NEURONIOS_C2, NUM_DADOS_DE_SAIDA)

E podemos checar os parâmetros internos dela!



In [8]:
for p in minha_MLP.parameters():
    print(p)

Parameter containing:
tensor([[-1.0288e-01, -2.6028e-01,  2.3473e-01, -1.5964e-01,  2.6214e-01,
          2.8054e-01],
        [-8.0346e-02, -3.0816e-01, -1.3232e-02, -3.7718e-01,  2.7229e-01,
         -1.2994e-01],
        [-1.6435e-01,  9.8518e-02, -4.0162e-01, -1.1341e-01,  2.9467e-01,
         -1.9066e-01],
        [ 3.2764e-01,  3.7803e-01, -2.3482e-01, -5.0682e-02, -2.5285e-01,
          1.1451e-01],
        [ 7.3917e-02,  3.3702e-01,  1.4937e-01, -1.8093e-01,  3.3858e-01,
         -3.2422e-02],
        [ 3.4440e-01,  1.3702e-01,  3.6554e-01,  2.4326e-01,  3.5221e-01,
          5.5392e-02],
        [ 3.1974e-02,  2.7481e-01, -7.2251e-02, -1.0799e-01,  3.6678e-01,
         -1.7341e-01],
        [ 3.4407e-01, -2.7447e-03,  7.5708e-02, -3.7938e-01, -1.1975e-02,
         -1.6102e-01],
        [-2.6364e-01, -1.8444e-01, -3.4000e-02,  3.0185e-01,  1.9220e-02,
          2.9813e-01],
        [ 1.7587e-01,  1.2335e-01, -3.2387e-01,  3.4601e-01,  3.0145e-01,
         -3.6938e-01],
        

Também podemos realizar uma previsão, mas ela provavelmente será bem ruim!



In [9]:
y_prev = minha_MLP(X_treino)
y_prev

tensor([[0.0450],
        [0.0491],
        [0.0510],
        ...,
        [0.0491],
        [0.0473],
        [0.0466]], grad_fn=<AddmmBackward>)

### A função de perda e o otimizador



O `pytorch` já vem com diversas funções de perda já implementadas. A que computa o erro quadrático médio, por exemplo, já está pronta!

Fora isso, precisamos definir o nosso otimizador. O otimizador é quem cuida de atualizar os parâmetros da nossa rede. Nós implementamos na nossa rede em Python puro a descida do gradiente. Aqui usaremos um otimizador que é uma modificação da descida do gradiente, chamado `Adam`. Trata-se, simplificadamente, de uma descida do gradiente com taxa de aprendizado individualizada para cada parâmetro e que se altera ao longo do aprendizado. `Adam` é um poderosíssimo otimizador! Não tem porque não usarmos ele neste caso.



In [10]:
TAXA_DE_APRENDIZADO = 0.001

# função perda será o erro quadrático médio
fn_perda = nn.MSELoss()

# otimizador será o Adam, um tipo de descida do gradiente
otimizador = optim.Adam(minha_MLP.parameters(), lr=TAXA_DE_APRENDIZADO)

### O treino da rede



Para avisar o `pytorch` que iremos treinar a rede, vamos invocar o método `train` da nossa rede. Toda rede recém-criada já está no modo treino, mas aqui deixarei explícito para ficar bem claro.



In [11]:
minha_MLP.train()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Observe que o ciclo do treino tem as mesmas etapas que vimos no notebook anterior, nada de novo! Aqui basta observar como executamos essas etapas no `pytorch`.



In [12]:
NUM_EPOCAS = 1000

y_true = y_treino

for epoca in range(NUM_EPOCAS):
    # forward pass
    y_pred = minha_MLP(X_treino)

    # zero grad
    otimizador.zero_grad()

    # loss
    loss = fn_perda(y_pred, y_true)

    # backpropagation
    loss.backward()

    # atualiza parâmetros
    otimizador.step()

    # mostra resultado
    print(epoca, loss.data)

0 tensor(0.0670)
1 tensor(0.0625)
2 tensor(0.0586)
3 tensor(0.0552)
4 tensor(0.0523)
5 tensor(0.0500)
6 tensor(0.0481)
7 tensor(0.0467)
8 tensor(0.0456)
9 tensor(0.0448)
10 tensor(0.0442)
11 tensor(0.0438)
12 tensor(0.0435)
13 tensor(0.0433)
14 tensor(0.0432)
15 tensor(0.0431)
16 tensor(0.0429)
17 tensor(0.0427)
18 tensor(0.0425)
19 tensor(0.0422)
20 tensor(0.0419)
21 tensor(0.0416)
22 tensor(0.0412)
23 tensor(0.0408)
24 tensor(0.0404)
25 tensor(0.0399)
26 tensor(0.0395)
27 tensor(0.0391)
28 tensor(0.0387)
29 tensor(0.0383)
30 tensor(0.0379)
31 tensor(0.0375)
32 tensor(0.0372)
33 tensor(0.0368)
34 tensor(0.0365)
35 tensor(0.0361)
36 tensor(0.0357)
37 tensor(0.0354)
38 tensor(0.0350)
39 tensor(0.0346)
40 tensor(0.0342)
41 tensor(0.0339)
42 tensor(0.0335)
43 tensor(0.0331)
44 tensor(0.0327)
45 tensor(0.0323)
46 tensor(0.0319)
47 tensor(0.0315)
48 tensor(0.0311)
49 tensor(0.0307)
50 tensor(0.0303)
51 tensor(0.0299)
52 tensor(0.0294)
53 tensor(0.0290)
54 tensor(0.0285)
55 tensor(0.0280)
56

Após o treino, podemos checar a performance da nossa rede. Observe a linha que inicia com `with`. Tudo que estiver no bloco do `with torch.no_grad()` é computado normalmente, porém o grafo computacional e os gradientes locais não são computados. Use isso sempre que for realizar contas com tensores fora do treino!!



In [13]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_treino)
    y_pred = minha_MLP(X_treino)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[1577.99996901] [1544.09722757]
[1409.99999118] [1536.49185562]
[1128.99997044] [1123.3394537]
[12991.99967575] [11635.80623245]
[14720.0001564] [14744.77986908]
[2938.99994564] [2681.14491463]
[6635.00014687] [8060.23932648]
[11215.99973297] [9599.58237457]
[1040.99999189] [1394.73832798]
[5040.00003624] [5263.28754997]
[9541.99988747] [8686.84072495]
[3792.99988461] [5272.91908264]
[1002.0000155] [778.87311459]
[2542.00005341] [1360.70108795]
[6788.00001717] [6687.83825111]
[9386.00025749] [8121.367836]
[1245.99996853] [1711.56492996]
[997.00000262] [473.90237093]
[708.00000262] [749.98733616]
[1435.99997544] [1459.59248996]
[2590.00006676] [2594.99456739]
[1392.99997497] [1713.12792492]
[11755.99946976] [9750.82494354]
[2007.99997377] [1257.87525749]
[2707.99998474] [2561.02567911]
[631.00000417] [634.20140839]
[652.99999881] [1062.05191612]
[2492.00006247] [2779.03343391]
[594.99999416] [603.93371201]
[760.0000056] [1141.48077869]
[1179.00003028] [1153.77031231]
[1789.9999361] [180

### O teste da rede



Vamos supor que você já testou diversas arquiteturas de rede e está pronto para usar os dados de teste. Nesta etapa precisamos indicar para o `pytorch` que o treino acabou e estamos na etapa de avaliação. Fazemos isso rodando o método `eval`.



In [14]:
minha_MLP.eval()

MLP(
  (camadas): Sequential(
    (0): Linear(in_features=6, out_features=50, bias=True)
    (1): ReLU()
    (2): Linear(in_features=50, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=1, bias=True)
  )
)

Agora podemos realizar o teste!



In [15]:
with torch.no_grad():
    y_true = normalizador_y.inverse_transform(y_teste)
    y_pred = minha_MLP(X_teste)
    y_pred = normalizador_y.inverse_transform(y_pred)

for yt, yp in zip(y_true, y_pred):
    print(yt, yp)

[523.00000858] [861.88162422]
[628.00000334] [606.00162792]
[442.00000328] [776.29448986]
[2284.99997044] [2681.30173779]
[1040.99999189] [839.48512745]
[809.99999654] [1271.34661484]
[3049.99987316] [3481.58133888]
[720.00000596] [579.25404644]
[877.99999249] [427.33429909]
[1292.00000429] [1662.7710166]
[7056.00004578] [4841.9450655]
[4176.99999142] [5855.08027077]
[13701.00020599] [15140.35016251]
[2948.0000515] [3984.71137238]
[16732.99985504] [14600.6679306]
[1815.00000048] [1891.24678612]
[16220.00016022] [11868.37525177]
[5967.00019073] [4756.66661644]
[473.00000042] [521.68561554]
[1078.9999795] [1137.7542305]
[452.00000319] [583.28762627]
[801.99998283] [834.768857]
[2458.99994993] [2600.56068707]
[2492.99998236] [2865.70344734]
[582.00000203] [681.00609303]
[1405.9999671] [1460.2015233]
[8715.99991035] [11465.67984009]
[7634.00004578] [6628.01448822]
[506.99999839] [636.61108303]
[665.99999094] [741.08204079]
[1815.00000048] [1570.03519917]
[707.00001383] [642.68391514]
[599.

E, finalmente, computar alguma métrica para medir a performance do nosso modelo.



In [16]:
RMSE = mean_squared_error(y_true, y_pred, squared=False)
print(f'Loss do teste: {RMSE}')

Loss do teste: 1386.1152778687401


## Conclusão

O objetivo deste notebook era de aprender a utilizar o módulo `pytorch` para criar e treinar uma rede neural. Este módulo é muito utilizado para esta finalidade por sua facilidade, simplicidade e ao mesmo tempo flexibilida, pois apesar de ser aplicado para um conceito dificil de rede neural, o módulo é de facil utilização, ja que todas as ferramentas necessária para a criação de redes neurais ja esta implementada dentro do modulo, contando com sua ciração e personalização, que é um dos fortes do módulo por conta de sua grande gama de possibilidades dos diferentes tipos de funções de ativação, tipos de rendes, etc. Outra grande vantagem do módulo é sua eficiência e otimização, pois o módulo foi feito e aprimorado das mais diversas formas.

Com isso, o módulo foi utilizado para a criação de uma rede neural com 6 inputs, 50 neurônios em sua primeira camada escondida, 20 em sua segunda camada escondida e 1 output. Esta rede foi treinada para tentar prever o preço de um diamante com a entrada de 6 dados. O resultado final foi um erro de aproximadamente 1386 dólares no set de teste, oque é, infelizmente, um valor médio para erro, considerando que os valores presentes no dataframe variam de 326 dólares até 18823 dólares. Apesar da rede não conseguir dizer precisamente com alta precisão o preço de um diamante ainda é possível ter um chute inicial para seu valor potencial. Mais importante, foi possível através do experimento desenvolvido no notebook, aprender a como utilizar o módulo pytorch para a aplicação de redes neurais para problemas, sendo um processo simples e sem muitas dificuldades.

## Playground

